<a href="https://colab.research.google.com/github/ductt-1167/Pytorch-Tutorial/blob/main/PytorchTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np 
import matplotlib.pyplot as plt
from time import time 
import torchvision
from torchvision import datasets, transforms 
import os
from google.colab import drive

In [55]:
import torch 
from torch import nn, optim

# Load Mnist dataset 

In [56]:
def prepare_dataloader(num_workers=8,
                       train_batch_size=64,
                       eval_batch_size=64):

    train_set = torchvision.datasets.MNIST(root="data",
                                             train=True,
                                             download=True,
                                             transform=transforms.ToTensor())
    
    val_set = torchvision.datasets.MNIST(root="data",
                                            train=False,
                                            download=True,
                                            transform=transforms.ToTensor())


    train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                               batch_size=train_batch_size,
                                               shuffle=True,
                                               num_workers=num_workers)

    val_loader = torch.utils.data.DataLoader(dataset=val_set,
                                             batch_size=eval_batch_size,
                                             shuffle=True,
                                             num_workers=num_workers)

    return train_loader, val_loader

In [57]:
trainloader, valloader = prepare_dataloader()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Plot image data

In [58]:
# dataiter = iter(trainloader)
# images, labels = dataiter.next()
# print(type(images))
# print(images.shape)
# print(labels.shape)
#
# plt.imshow(images[0].numpy().squeeze(), cmap='gray_r');
#
# figure = plt.figure()
# num_of_images = 60
# for index in range(1, num_of_images + 1):
#     plt.subplot(6, 10, index)
#     plt.axis('off')
#     plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')

# Build the network

https://pytorch.org/docs/stable/nn.html 

In [59]:
class MyNet(nn.Module):
	def __init__(self, num_channels, classes):
    # call the parrent constructor 
		super(MyNet, self).__init__()

    # Init layers: Conv, ReLU, Pool.
		self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=20, kernel_size=(5, 5))
		self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5))

		self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		self.fc1 = nn.Linear(in_features=800, out_features=500)
		self.fc2 = nn.Linear(in_features=500, out_features=classes)

		self.relu = nn.ReLU()
		self.log_soft = nn.LogSoftmax(dim=1)

	def forward(self, x):
		# pass the input through our first set of CONV => RELU => POOL layers
		x = self.conv1(x)
		x = self.relu(x)
		x = self.pool1(x)

		# pass the output from the previous layer through the second set of CONV => RELU => POOL layers
		x = self.conv2(x)
		x = self.relu(x)
		x = self.pool2(x)

		# flatten the output from the previous layer and pass it through our only set of FC => RELU layers
		x = torch.flatten(x, 1)
		x = self.fc1(x)
		x = self.relu(x)

		# pass the output to our softmax classifier to get our output predictions
		x = self.fc2(x)
		output = self.log_soft(x)

		# return the output predictions
		return output

# Training model

In [60]:
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
# define training hyperparameters
INIT_LR = 1e-3
EPOCHS = 10
BATCH_SIZE = 64

model = MyNet(num_channels=1, classes=len(trainloader.dataset.classes)).to(device)
opt = torch.optim.Adam(model.parameters(), lr=INIT_LR)
loss_fn = nn.CrossEntropyLoss()

# calculate steps per epoch for training and validation set
train_steps = len(trainloader.dataset) // BATCH_SIZE
val_steps = len(valloader.dataset) // BATCH_SIZE


In [62]:
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}

# measure how long training is going to take
print("[INFO] training the network...")
start_time = time()

[INFO] training the network...


In [ ]:
# loop over our epochs
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()

	# initialize the total training and validation loss
	total_train_loss = 0
	total_val_loss = 0

	# initialize the number of correct predictions in the training and validation step
	train_correct = 0
	val_correct = 0

	# loop over the training set
	for i, (x, y) in enumerate(trainloader):
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))

		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = loss_fn(pred, y)

		# zero out the gradients, perform the backpropagation step, and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()

		# add the loss to the total training loss so far and calculate the number of correct predictions
		total_train_loss += loss
		train_correct += (pred.argmax(1) == y).type(
			torch.float).sum().item()
   
  # switch off autograd for evaluation
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()

		# loop over the validation set
		for i, (x, y) in enumerate(valloader):
			# send the input to the device
			(x, y) = (x.to(device), y.to(device))

			# make the predictions and calculate the validation loss
			pred = model(x)
			total_val_loss += loss_fn(pred, y)

			# calculate the number of correct predictions
			val_correct += (pred.argmax(1) == y).type(
				torch.float).sum().item()
    
  # calculate the average training and validation loss
	avg_train_loss = total_train_loss / train_steps
	avg_val_loss = total_val_loss / val_steps

	# calculate the training and validation accuracy
	train_correct = train_correct / len(trainloader.dataset)
	val_correct = val_correct / len(valloader.dataset)

	# update our training history
	H["train_loss"].append(avg_train_loss.cpu().detach().numpy())
	H["train_acc"].append(train_correct)
	H["val_loss"].append(avg_val_loss.cpu().detach().numpy())
	H["val_acc"].append(val_correct)

	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avg_train_loss, train_correct))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avg_val_loss, val_correct))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[INFO] EPOCH: 1/10
Train loss: 0.155060, Train accuracy: 0.9528
Val loss: 0.041959, Val accuracy: 0.9867

[INFO] EPOCH: 2/10
Train loss: 0.046196, Train accuracy: 0.9855
Val loss: 0.033349, Val accuracy: 0.9890

[INFO] EPOCH: 3/10
Train loss: 0.030136, Train accuracy: 0.9902
Val loss: 0.024930, Val accuracy: 0.9919

[INFO] EPOCH: 4/10
Train loss: 0.022874, Train accuracy: 0.9927
Val loss: 0.026712, Val accuracy: 0.9910

[INFO] EPOCH: 5/10
Train loss: 0.018057, Train accuracy: 0.9944
Val loss: 0.024304, Val accuracy: 0.9908

[INFO] EPOCH: 6/10
Train loss: 0.013904, Train accuracy: 0.9955
Val loss: 0.028141, Val accuracy: 0.9910

[INFO] EPOCH: 7/10
Train loss: 0.012330, Train accuracy: 0.9959
Val loss: 0.030297, Val accuracy: 0.9897

[INFO] EPOCH: 8/10
Train loss: 0.009916, Train accuracy: 0.9967
Val loss: 0.036942, Val accuracy: 0.9910

[INFO] EPOCH: 9/10
Train loss: 0.008654, Train accuracy: 0.9974
Val loss: 0.029325, Val accuracy: 0.9917



In [ ]:
# finish measuring how long training took
end_time = time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	end_time - start_time))